In [2]:
# !pip install neo4j
import neo4j
import pandas as pd

In [84]:
def connect_db():
    driver = neo4j.GraphDatabase.driver(uri="neo4j://0.0.0.0:7687", auth=("neo4j","password"))
    session = driver.session(database="neo4j")
    return session
    
def wipe_out_db(session):
    # wipe out database by deleting all nodes and relationships
    
    # similar to SELECT * FROM graph_db in SQL
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

session = connect_db()
wipe_out_db(session) 

In [65]:
def run_query_to_pandas(session, query):
    # run a query and return the results in a pandas dataframe
    
    result = session.run(query)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

def run_query_print_raw(session, query):
    result = session.run(query)
    
    for r in result:
        print(r.values())

#### Data Insertion 

In [85]:
query = """

CREATE
    
    // Insert User Nodes //
    (Qi:User{username: 'qlin1', first_name: 'Qi', last_name: 'L', email: 'qlin1@vandy.edu', class:2024, company: 'Deloitte', location: 'New York', interests: ['cooking', 'finance', 'chess']}),
    (Sam:User{username: 'sam1', first_name: 'Sam', last_name: 'A', email: 'sam1@vandy.edu', class:2024, company: 'Deloitte', location: 'Chicago', interests: ['travel', 'music', 'biking']}),
    (Promod:User{username: 'prom1', first_name: 'Promod', last_name: 'R', email: 'prom1@vandy.edu', class:2024, company: 'Barclays', location: 'New York', interests: ['finance', 'gardening', 'gaming']}),
    (Ji:User{username: 'ji1', first_name: 'Ji', last_name: 'N', email: 'ji1@vandy.edu', class:2024, company: 'Adobe', location: 'Austin', interests: ['photography', 'cooking', 'hiking']}),
    (Minu:User{username: 'minu1', first_name: 'Minu', last_name: 'S', email: 'minu1@vandy.edu', class:2024, company: 'DSI', location: 'Nashville', interests: ['golf', 'LoL', 'cooking']}),
    (John:User{username: 'john1', first_name: 'John', last_name: 'P', email: 'john1@vandy.edu', class:2024, company: 'DSI', location: 'Nashville', interests: ['golf', 'LoL', 'cooking']}),

    // Insert Post Nodes //
    (post1:Post{post_id: 1, author: "qlin1", topic: "General", timestamp: datetime(), content: 'Hello World!', date: '2023-11-09', likes_count: 0, comments_count: 0}),
    (post2:Post{post_id: 2, author: "sam1", topic: "General", timestamp: datetime(), content: 'Hello NoSQL!', date: '2023-11-09', likes_count: 0, comments_count: 0}),
    (post3:Post{post_id: 3, author: "prom1", topic: "General", timestamp: datetime(), content: 'Hello Graph!', date: '2023-11-09', likes_count: 0, comments_count: 0}),

    // Insert Message Nodes //
    (msg1:Message{message_id: 1, sender: "qlin1", receiver: "sam1", timestamp: datetime(), content: 'Hello Sam!'}),
    (msg2:Message{message_id: 2, sender: "sam1", receiver: "qlin1", timestamp: datetime(), content: 'Hello Qi!'}),
    (msg3:Message{message_id: 3, sender: "ji1", receiver: "qlin1", timestamp: datetime(), content: 'Hello Qi!'}),

    // Insert Group Node // 
    (group1:Group{group_name: "Class 2024", description: "DSI cohort 2024", members_count: 38, created_by: "qlin1"}),

    // Insert edge between user and user //
    (Qi)-[:FRIENDS_WITH{since: 2022}]->(Sam),
    (Qi)-[:FRIENDS_WITH{since: 2022}]->(Promod),
    (Qi)-[:FRIENDS_WITH{since: 2022}]->(Ji),
    (Sam)-[:FRIENDS_WITH{since: 2022}]->(Promod),
    (Sam)-[:FRIENDS_WITH{since: 2022}]->(Ji),
    (Promod)-[:FRIENDS_WITH{since: 2022}]->(Ji),
    (Minu)-[:FRIENDS_WITH{since: 2022}]->(Ji),

    // Insert edge between user and post //
    (Qi)-[:POSTED{on_date: date('2023-11-09')}]->(post1),
    (Sam)-[:POSTED{on_date: date('2023-11-09')}]->(post2),
    (Promod)-[:POSTED{on_date: date('2023-11-09')}]->(post3),

    // Insert edge between user and message //
    (Qi)-[:SENT_MESSAGE {on_date: date("2023-11-09")}]->(msg1),
    (Sam)-[:SENT_MESSAGE {on_date: date("2023-11-09")}]->(msg2),
    (Ji)-[:SENT_MESSAGE {on_date: date("2023-11-09")}]->(msg3),
    (Qi)-[:RECEIVED_MESSAGE {on_date: date("2023-11-09")}]->(msg2),
    (Qi)-[:RECEIVED_MESSAGE {on_date: date("2023-11-09")}]->(msg3),

    // Insert edge between user and group //
    (Qi)-[:MEMBER_OF {since: date("2023-11-09")}]->(group1),
    (Sam)-[:MEMBER_OF {since: date("2023-11-09")}]->(group1),
    (Promod)-[:MEMBER_OF {since: date("2023-11-09")}]->(group1),
    (Ji)-[:MEMBER_OF {since: date("2023-11-09")}]->(group1)

"""
session.run(query)

#### Data Retrieval & Querying

In [21]:
# Retrieve a property of a specific User 

query = '''
    MATCH (u: User)
    WHERE u.first_name = 'Qi'
    RETURN u.first_name + ' ' + u.last_name AS full_name, u.email as Email, u.class as Class, u.company as Company, u.location as Location
'''

df = run_query_to_pandas(session, query)
display(df)

,full_name,Email,Class,Company,Location
0,Qi L,qlin1@vandy.edu,2024,Deloitte,New York


In [24]:
# Find all Posts created by a specific User
query = '''
    MATCH (u: User)-[:POSTED]->(p: Post)
    WHERE u.first_name = 'Qi'
    RETURN u.first_name + ' ' + u.last_name AS full_name, p.content as Content, p.date as Date, p.likes_count as Likes, p.comments_count as Comments
'''
df = run_query_to_pandas(session, query)
display(df)

,full_name,Content,Date,Likes,Comments
0,Qi L,Hello World!,2023-11-09,0,0


In [36]:
# Find all Users who posted a specific topic of Post (topic is a property of Post)
query = '''
    MATCH (u: User)-[:POSTED]->(p: Post)
    WHERE p.topic = 'General'
    RETURN u.first_name + ' ' + u.last_name AS full_name, p.topic as Topic, p.content as Content, p.date as Date, p.likes_count as Likes, p.comments_count as Comments
'''
df = run_query_to_pandas(session, query)
display(df)

,full_name,Topic,Content,Date,Likes,Comments
0,Qi L,General,Hello World!,2023-11-09,0,0
1,Sam A,General,Hello NoSQL!,2023-11-09,0,0
2,Promod R,General,Hello Graph!,2023-11-09,0,0


In [64]:
# Find common interests between two specific Users. 
query = '''
    MATCH (user1:User {username: 'qlin1'}), (user2:User {username: 'ji1'})
    RETURN user1.first_name + ' ' + user1.last_name AS User1, 
        user2.first_name + ' ' + user2.last_name AS User2, 
        [val IN user1.interests WHERE val IN user2.interests] AS common_interests
'''
df = run_query_to_pandas(session, query)
display(df)


,User1,User2,common_interests
0,Qi L,Ji N,[cooking]


In [25]:
# Retrieve top 3 Users who created most Posts
query = '''
    MATCH (u: User)-[:POSTED]->(p: Post)
    RETURN u.first_name + ' ' + u.last_name AS full_name, count(p) as Posts
    ORDER BY Posts DESC
    LIMIT 3
'''
df = run_query_to_pandas(session, query)
display(df)

,full_name,Posts
0,Qi L,1
1,Sam A,1
2,Promod R,1


In [31]:
# Retrieve Users who haven’t created any Posts
query = '''
    MATCH (u: User)
    WHERE NOT (u)-[:POSTED]->(: Post)
    RETURN u.first_name + ' ' + u.last_name AS full_name
'''
df = run_query_to_pandas(session, query)
display(df)

,full_name
0,Ji N


In [83]:
# Given two Users, identify if they are indirectly connected through a chain of friends and, if so, return the connecting path. 
# For instance, can you find a relationship path where Alice is friends with someone (say, Bob), who in turn is friends with Charlie?
query = '''
MATCH path = (startUser:User {username: 'qlin1'})-[:FRIENDS_WITH*2]-(endUser:User {username: 'minu1'})
RETURN [node IN nodes(path) | node.first_name] AS Connecting_Path
'''
df = run_query_to_pandas(session, query)
display(df)

,Connecting_Path
0,"[Qi, Ji, Minu]"


In [87]:
# Write a query to identify orphaned Users (Users who have no connections). 
# Can you think of a scenario when this information is useful?
query = '''
    MATCH (u: User)
    WHERE NOT (u)-[:FRIENDS_WITH]-(: User)
    RETURN u.first_name + ' ' + u.last_name AS full_name
'''
df = run_query_to_pandas(session, query)
display(df)

# Answer: Identifying orphaned users can help the network manager reach out and engage these users to increase their connection within the community.
# Beaus orphaned users may be at higher risk of leaving the service due to a lack of engagement. 
# Identifying these users can help implement user retention strategies targeted at increasing their network and engagement.

,full_name
0,John P
